In [1]:
from functools import partial
import paddle
from paddlenlp.datasets import load_dataset
from paddlenlp.transformers import AutoModelForSequenceClassification, AutoTokenizer
from paddlenlp.trainer import Trainer, TrainingArguments


/home/ricardo/miniconda3/envs/PaddleEE/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_dataset = load_dataset("chnsenticorp", splits=["train"])
model = AutoModelForSequenceClassification.from_pretrained("ernie-1.0-base-zh", num_classes=len(train_dataset.label_list))
tokenizer = AutoTokenizer.from_pretrained("ernie-1.0-base-zh")

def convert_example(example, tokenizer):
    encoded_inputs = tokenizer(text=example["text"], max_seq_len=128, pad_to_max_seq_len=True)
    encoded_inputs["labels"] = int(example["label"])
    return encoded_inputs

train_dataset = train_dataset.map(partial(convert_example, tokenizer=tokenizer))

[2022-05-23 23:54:04,523] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.modeling.ErnieForSequenceClassification'> to load 'ernie-1.0-base-zh'.
[2022-05-23 23:54:04,524] [    INFO] - Already cached /home/ricardo/.paddlenlp/models/ernie-1.0-base-zh/ernie_v1_chn_base.pdparams
[2022-05-23 23:54:14,933] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load 'ernie-1.0-base-zh'.
[2022-05-23 23:54:14,934] [    INFO] - Already cached /home/ricardo/.paddlenlp/models/ernie-1.0-base-zh/vocab.txt


In [3]:
print(train_dataset[100])

[2022-05-23 23:54:15,043] [    INFO] - The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
training_args = TrainingArguments('./output')
training_args.do_train = True
training_args.evaluation_strategy = 'steps'
# print(training_args)

In [4]:
class Custom_MSE(paddle.nn.Layer):
    def __init__(self):
        super(Custom_MSE, self).__init__()

    def forward(self, predictions, target):
        square_difference = paddle.square(predictions - target)
        loss_value = paddle.mean(square_difference)
        return loss_value
    
    # def __call__(self, predictions, target):
    #   square_difference = paddle.square(y_predictions - target)
    #   loss_value = paddle.mean(square_difference)
    #   return loss_value

In [5]:
# trainer = Trainer(
#     model=model,
#     criterion=paddle.nn.loss.CrossEntropyLoss(),
#     args=training_args,
#     train_dataset=train_dataset if training_args.do_train else None,
#     tokenizer=tokenizer)

# if training_args.do_train:
#     train_result = trainer.train()
#     metrics = train_result.metrics
#     trainer.save_model()
#     trainer.log_metrics("train", metrics)
#     trainer.save_state()